# **Preprocesing**

### **Exploring Dataset**

Dalam tahapan ini kita akan melakukan importing beberapa asset yang dibutuhkan seperti library yang dibutuhkan, dataset yang dibutuhkan

In [5]:
# import library yang dibutuhkan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from wordcloud.wordcloud import WordCloud


In [6]:
# untuk mengimport dataset
df = pd.read_csv("chatgpt_reviews.csv")

# untuk melihat isi data
df.head()

,date,title,review,rating
0,2023-05-21 16:42:24,Much more accessible for blind users than the ...,Up to this point I’ve mostly been using ChatGP...,4
1,2023-07-11 12:24:19,"Much anticipated, wasn’t let down.",I’ve been a user since it’s initial roll out a...,4
2,2023-05-19 10:16:22,"Almost 5 stars, but… no search function",This app would almost be perfect if it wasn’t ...,4
3,2023-05-27 21:57:27,"4.5 stars, here’s why","I recently downloaded the app and overall, it'...",4
4,2023-06-09 07:49:36,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4


setelah itu kita akan melakukan perhitungan ada berapa banyak review untuk masing masing rating 

In [7]:
# untuk menghitung nilai unik
df["rating"].value_counts()

rating
5    1134
1     495
4     304
3     220
2     139
Name: count, dtype: int64

berdasarkan dari data di atas, kita dapat menyimpulkan untuk komentar positif itu pada rating 5 dan untuk komentar negatif itu pada rating 1. Mengapa hanya rating 1 dan 5 ? karena kalau komentar 2 - 4 itu masih termasuk pendapat. 

Dan juga kita dapat melihat banyaknya data yang rating 5 dengan data dengan rating 1 sangatlah berbeda jauh perbedaannya, maka kita perlu menyeimbangkan data ini untuk menghindari potensi overfitting

### **Feature Selection**

dalam tahapan ini kita akan memilih fitur yang berguna saja. Jadi kita dapat membuang fitur yang tidak digunakan. Dalam kasus ini kita hanya membutuhkan text komentar sebagai x nya dan label (rating) sebagai y nya

In [8]:
# menghapus fitur
df.drop(columns=["date", "title"], inplace=True)

df

,review,rating
0,Up to this point I’ve mostly been using ChatGP...,4
1,I’ve been a user since it’s initial roll out a...,4
2,This app would almost be perfect if it wasn’t ...,4
3,"I recently downloaded the app and overall, it'...",4
4,I appreciate the devs implementing Siri suppor...,4
...,...,...
2287,This has to be a beginning to something crazy ...,5
2288,I’ve been using chat and have been a proud pre...,5
2289,The ChatGPT iOS app is an outstanding product....,5
2290,Sam Altman’s blatant attempt at regulatory cap...,2


### **Duplicated Data**

pada tahapan ini kita akan menghapuskan untuk data yang tak diperlukan lagi. Tujuannya adalah agar tidak terjadi nya pembelajaran yang sama pada model kita. Jika hal ini dipaksakan, hal ini akan berpotensi terjadinya overfitting

In [9]:
# menampilkan data yang terduplikasi
df[df.duplicated()]

,review,rating
1351,Thanks,5
1508,Please,5
1783,Love it,5
1800,First,5
1801,First,5
1802,First,5
1827,Why?,1
1853,Very good,5
1866,Pretty good,5
1869,Great,5


dari tampilan di atas sangat jelas kalau ada data yang terduplikasi. Maka kita perlu membiarkan satu data saja pada data yang terduplikasi tersebut

In [10]:
# menghilangkan data terduplikasi
df.drop_duplicates(inplace=True)

# melihat apakah ada data yang terduplikasi
df[df.duplicated()]

,review,rating


### **Missing Value**

Pada tahapan ini kita akan mencari apakah ada data yang kosong atau tidak. Hal ini bertujuan untuk menghindari terjadinya error pada proses modeling dan juga menghindari kesalah pahaman pada model (misal Nan akan dianggap dokumen yang berisi kata Nan)

In [11]:
# melihat missing jenis Nan
df.isna().sum()

review    0
rating    0
dtype: int64

In [12]:
# melihat missing jenis null
df.isnull().sum()

review    0
rating    0
dtype: int64

Tidak ditemukan data yang hilang, maka kita bisa mengabaikannya saja

setelah dilakukan cleaning, maka kita perlu menyeimbangkan banyaknya data agar tidak menjadi overfitting

In [13]:
# mengambil data yang diperlukan
df = pd.concat([df[df["rating"] == 5].iloc[:500], df[df["rating"] == 1]])\
    .sample(frac=1, random_state=123)

# menampilkan data
df.head(10)

,review,rating
282,Can’t thank the OpenAI Team enough for the bea...,5
709,Surprised apple let this through its approval ...,1
2272,Beware!!! This app requires personal info to u...,1
2251,It's not sending me the code to verify my number,1
293,The UI is clean and the haptic feedback is app...,5
1378,Thinks my cell number is a landline. No way ar...,1
1404,I updated chatgpt but it wont answer to my que...,1
701,"Hello, do you guys have a timeline of when wou...",5
555,This Mobile App works very well and is effecti...,5
456,This app is an absolute game changer and I abs...,5


### **Train Test Split**

Pada tahapan ini kita akan memisahkan data yang hanya digunakan untuk test (data test) dan data yang digunakan untuk training (data train). Tujuan dari splitting ini adalah agar kita mengetahui performa dari model kita

In [14]:
# mengembalikan data yang sudah di split train dengan test
def Train_Test_Split(x, y, test_size=0.2, random_seed=None):
    # banyaknya data
    n = len(x)

    # jika ada random seed
    if random_seed:
        np.random.seed(random_seed)

    # menentukan banyaknya data test
    test_size = int(test_size * n)

    # matrix random
    indices = np.random.permutation(n)

    # menentukan train dan test 
    train_indices, test_indices = indices[test_size:], indices[:test_size]
    return x.iloc[train_indices], x.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

# menentukan x dan y nya
x = df["review"]
y = df["rating"]

# mengimplementasikan pada dataset
X_train, X_test, y_train, y_test = Train_Test_Split(x, y, test_size=0.2, random_seed=123)

X_train

1525            This app is asking for your phone number.
2002          有病吧，突破重重困难才下载了它，满怀欣喜得去登录，居然不给过，那又何必开放这破软件呢？
823     I pay for plus and can access browsing and plu...
37      I don’t typically write review but this offici...
1832    It agrees with letting children be forced into...
                              ...                        
171     Please make the ipad version fit the screen. C...
655     Works as it should but drains battery fast and...
1656                          Fails right out of the box.
1498                  Heating problems when ask something
1355    Simple as that. It works about 1 out of 7 time...
Name: review, Length: 796, dtype: object

### **Case Folding**

Pada tahapan ini kita akan melakukan seleksi dalam kata atau karakter yang mengganggu seperti tag, hastag, emoji, tanda baca, dan lain lain akan kita hapus. Karena karakter seperti ini akan mengganggu dalam makna suatu kata

In [26]:
# mengembalikan text yang sudah di case
def case_folding(text):
    # untuk menghilangkan tag "@"
    text = re.sub(r"@[A-Za-z0-9_]+", "", text)

    # untuk menghilangkan hashtag "#"
    text = re.sub(r"#\w+", "", text)

    # untuk menghilangkan new line atau enter
    text = re.sub(r"\n+", "", text)

    # untuk menghilangkan karakter angka
    text = re.sub(r"\d+", "", text)

    # untuk menghilangkan karakter RT
    text = re.sub(r"RT[\s]+", "", text)

    # untuk menghilangkan link
    text = re.sub(r"https?://\S+", "", text)
    
    # untuk menghilangkan tanda baca
    text = text.translate(str.maketrans("", "", string.punctuation))

    # untuk menghilangkan emoji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r"", text)

    # mengembalikan bentuk string yang sudah huruf kecil 
    return text.lower()
    

# men case folding untuk setiap dokumen
X_train = X_train.apply(case_folding)

X_train

1525             this app is asking for your phone number
2002          有病吧，突破重重困难才下载了它，满怀欣喜得去登录，居然不给过，那又何必开放这破软件呢？
823     i pay for plus and can access browsing and plu...
37      i don’t typically write review but this offici...
1832    it agrees with letting children be forced into...
                              ...                        
171     please make the ipad version fit the screen cu...
655     works as it should but drains battery fast and...
1656                           fails right out of the box
1498                  heating problems when ask something
1355    simple as that it works about  out of  times w...
Name: review, Length: 796, dtype: object

In [27]:
# sebelum dilakukan case folding
df["review"].loc[121]

"For now, I only managed to get it using the US store. This is a game-changer for how we interact with AI technology. 📱\n\nThe app syncs your history across devices and includes the latest model improvements. What's more, it has integrated Whisper, OpenAI's open-source speech-recognition system, for voice input, and a cool haptic response for when an answer is generated. This is a huge step forward in making AI more accessible and user-friendly. 🗣️"

In [28]:
# setelah dilakukan case folding
X_train.loc[121]

'for now i only managed to get it using the us store this is a gamechanger for how we interact with ai technology the app syncs your history across devices and includes the latest model improvements whats more it has integrated whisper openais opensource speechrecognition system for voice input and a cool haptic response for when an answer is generated this is a huge step forward in making ai more accessible and userfriendly ️'

### **Tokenize**

Pada tahapan ini kita akan melakukan pemisahan text menjadi per kata. Jadi kita akan jadikan satu dokumen memiliki list yang berisi kata. Kata ini lah yang nantinya akan jadi fitur

In [30]:
# akan mengembalikan text yang sudah ditokenize
def tokenize(text):
    return word_tokenize(text)

# mengimplemantasikan untuk setiap dokumen
X_train = X_train.apply(tokenize)

X_train

1525    [this, app, is, asking, for, your, phone, number]
2002        [有病吧，突破重重困难才下载了它，满怀欣喜得去登录，居然不给过，那又何必开放这破软件呢？]
823     [i, pay, for, plus, and, can, access, browsing...
37      [i, don, ’, t, typically, write, review, but, ...
1832    [it, agrees, with, letting, children, be, forc...
                              ...                        
171     [please, make, the, ipad, version, fit, the, s...
655     [works, as, it, should, but, drains, battery, ...
1656                    [fails, right, out, of, the, box]
1498            [heating, problems, when, ask, something]
1355    [simple, as, that, it, works, about, out, of, ...
Name: review, Length: 796, dtype: object

### **Stopwords**

Pada tahapan stopwords ini kita akan melakukan seleksi kata lagi. Untuk kata yang memiliki makna dependent akan dihapus, karena kata di dalamnya harus independent. Hal ini disebabkan pembobotan kata akan melibatkan satu kata menjadi satu term / feature

In [37]:
# kata yang harus dihilangkan
stopword_var = stopwords.words()

# menghilangkan kata yang harus dihilangkan
def stopword(doc):
    return [text for text in doc if text not in stopword_var]

# mengimplementasikan untuk setiap dokumen
X_train = X_train.apply(stopword)

X_train

1525                                 [app, phone, number]
2002        [有病吧，突破重重困难才下载了它，满怀欣喜得去登录，居然不给过，那又何必开放这破软件呢？]
823     [pay, access, browsing, plugins, search, app, ...
37      [’, typically, write, review, official, app, o...
1832    [agrees, letting, children, forced, pride, hom...
                              ...                        
171     [make, ipad, version, fit, screen, currently, ...
655          [works, drains, battery, fast, heats, phone]
1656                                         [fails, box]
1498                                  [heating, problems]
1355           [simple, works, times, webpage, work, app]
Name: review, Length: 796, dtype: object

### **Stemming**

Pada tahapan ini kita akan melakukan pengubahan pada kata yang bukan kata asli menjadi kata aslinya. Yang ada kata imbuhannya menjadi kata yang asli. Hal ini bertujuan agar suatu kata memiliki makna yang sama itu akan tetap dianggap sama

In [38]:
# fungsi yang akan melakukan stem
stemmer = PorterStemmer()

# melakukan stemming setip dokumen
def stemming(doc):
    return [stemmer.stem(text) for text in doc]

# implementasikan untuk setiap dokumen
X_train = X_train.apply(stemming)

X_train

1525                                 [app, phone, number]
2002        [有病吧，突破重重困难才下载了它，满怀欣喜得去登录，居然不给过，那又何必开放这破软件呢？]
823     [pay, access, brows, plugin, search, app, make...
37      [’, typic, write, review, offici, app, openai,...
1832         [agre, let, children, forc, pride, homosexu]
                              ...                        
171     [make, ipad, version, fit, screen, current, ’,...
655             [work, drain, batteri, fast, heat, phone]
1656                                          [fail, box]
1498                                      [heat, problem]
1355               [simpl, work, time, webpag, work, app]
Name: review, Length: 796, dtype: object

# **Pembobotan Kata**

### **Analize Frequency Words**

### **Document Frequency**

### **Term Frequency**

### **Inverse Document Frequency**

### **TF - IDF**

### **Feature Selection**

# **Modeling**

### **Create Class**

### **Training**

### **Testing**

### **Conclusion**